**The Big Picture**

The basic idea of RSA is that it is relatively easy to take two prime numbers and multiply them together. It is also relatively easy to raise a number to an exponent and find the modulo of that number for any given base. It is _not_ however so easy to go the other way and determine what two prime numbers were mulitplied together. That is the basic idea behind RSA; your data is safe so long as it takes "too long" for someone else to compute what two numbers you used.

**How does any of this work?**

Let's start with the idea of modulo arithmatic since this is the basis of what keeps your data secure. Alex Morrow has created a library called **modulararithmatic.py** which we will use to explore this topic

In [1]:
import numpy as np
import modulararithmetic as ma

The basic idea of modulo arithmatic is fairly straighforward. Imagine a circle of numbers, such as a clock, where all operations wrap back around. For example: take the arithmatic operation of 2 + 3. If your clock has only 4 numbers - you are working (mod 4) - you would start at 2, take 3 steps forward and wind up wrapping back around to 1. Likewise, if you started with the value (2+3)(mod4), it would be represented the same way because modular arithmatic does not depend on the order operations are performed. Let's take a moment to explore how this is implemented in **moduloarithmatic.py**.

In [18]:
a = ma.mod(5)(7)
print(int(a))

#For addition (mod n), the order of operations does not affect the result
b = ma.mod(5)(6+8)
c = ma.mod(5)(6) + ma.mod(5)(8)
print(int(b) == int(c))
print(int(b))

#If you have created two ModX objects with the same (mod n), all normal arithmatic operations will be perfomred (mod n)
print(int(a+b))


#Find 15 (mod 7)

#Find 23456 (mod 123)

#Find [17 (mod9) + 35 (mod9)] (mod 7)

2
True
4
1


Now that you have a little more understanding of how to find the modulo of a number using this library let's start to get to 
some more interesting operations like multiplication. 

In [24]:
a = ma.mod(5)(7)
b = ma.mod(5)(18)
print(int(a*b))
#Try multiplying any two numbers base 24



1


So now that you have a bit of an idea of how addition and multiplication work (mod n) Let's talk a little bit about inverses, because those are going to be very important to understanding RSA. In normal math a number times its inverse will result in 1 and a number plus its inverse will result in 0. These numbers 1 and 0 are called identities because if you multiply/ add -respectively - any number by them the result will be the same one you put in. In modulo arithmatic the same is ture, however instead of using negative numbers or fractions we have to rely on the numbers from 1 to n. So let's explore that a little bit in regards to multiplication.

In [30]:
#To find the inverse of a ModX object (mod n) use: - (addition) or ~ (multiplication)
a = ma.mod(5)(7)
b = ~a
print(int(b))
b = ma.mod(5)(18)
c = -b
print(c)

#find the inverse of b = ma.mod(5)(19)


3
'2'


Awesome! Now that you have the basics of using modular arithmatic let's start looking at how this applies to RSA. One of the primary ideas that RSA is based on is the idea that given a number (mod n) we can find the inverse so it becomes very important to choose n so that there will in fact be an inverse. Let's start exploring how to choose *n* by looking at what happens when n is equal to 6.

In [33]:
#for every number (mod 6) print the multiplicative inverse
n = 6
for i in range(1, n):
    a = ma.mod(n)(i)
    try:
        print('{} is the inverse of {}'.format(~a, i))
    except:
        print('{} is not invertable'.format(i))

#Change the code above to find sets of numbers where every value has an inverse Once you have found three that work,
#what do they have in common?

'1' is the inverse of 1
2 is not invertable
3 is not invertable
4 is not invertable
'5' is the inverse of 5


It turns out, if *n* is prime, you can also get back to 1 by exponentiation. Let's explore that for a moment.

In [92]:
output = np.array([])
n = 3
for e in range(1, n):
    output = np.append(output, 'e = {} |'.format(e))
    for i in range(1, n):
        output = np.append(output, int(ma.mod(n)(i**e)))
output = output.reshape(n-1, n)
print(output)

#Try changing n to other prime numbers and see if you can find a pattern for what numbers are equal to zero

[['e = 1 |' '1' '2']
 ['e = 2 |' '1' '1']]


So what you may have just noticed is that the bottom row, where the exponent is equal to n-1, always appears to be 1. This idea is actually what makes up Fermot's little theorum and holds true for any prime number. Another way to look at this property is Euler's totient which looks at the number of fractions in simplest form that can be madeby placing n as the denomenator and cycling through number as the numerator. 

In [105]:
import euler as eu
euler = eu.Euler()
n = 6
for i in range(1, n+1):
    print('{}/{}'.format(i,n))
    
#Using Alex Morrow's euler.py we can verify the totient
print('The totient is {}'.format(euler.totient(n)))

<euler.Euler instance at 0x7fa880502ef0>.__init__((),{})
init primetools
1/6
2/6
3/6
4/6
5/6
6/6
The totient is 2


For the example given, the totient is 2. However, if the number is prime, none of the fractions it creates will reduce and the totient will always be n-1. 

So this may seem unrelated, but we are actually getting into some of the heart of what makes up RSA. If you recall from the introduction (which was admitedly a while ago) the public key is made up of two prime numbers multiplied together. So far we have only been dealing with a single prime number, but there is another method that can be used when two prime numbers are multiplied together, it turns out that the totient will be equal to the totient of the first times the totient of the second. In other words, the totient of _pq_ is equal to _(p-1)(q-1)_. Let's try that out.

In [106]:
p = 5
q = 3
t_guess = (p-1)*(q-1)
t = euler.totient(p*q)
print('(p-1)(q-1) = {}'.format(t_guess))
print('euler.py evaluation = {}'.format(t))

(p-1)(q-1) = 8
euler.py evaluation = 8


Armed with this knowledge of modular arithmatic and the importance of primes, we can get started with how to actually implement RSA and why it works at all.